# TP SVM Novembre 2017

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
# importer les données
from sklearn.datasets import load_iris
iris = load_iris()

In [ ]:
# afficher la description de la base de données
print(iris.DESCR)

In [ ]:
# iris.feature_names[]
iris.feature_names

### SVM à noyau linéaire
Nous allons nous limiter pour l'instant à deux classes : setosa et virginica, et à deux features (pour visualiser) : sepal length and sepal width. Nous allons entrainer une SVM linéaire et afficher l'hyperplan séparateur (en 2D une droite donc).

Nous allons utiliser la classe [SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) du module svm de scikit-learn.

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear', C=1000)

# on sélectionne les données voulues (2 classes et 2 features)
X = iris.data[iris.target!=1, :2]
print(X.shape)
y = iris.target[iris.target!=1]
print(y.shape)
clf.fit(X, y)

#### Représentons l'hyperplan séparateur !

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# plot support vectors
#ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
#           linewidth=1, facecolors='none')

ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=30,
           linewidth=1, marker='x', color='k')

__Question :__ Où sont situés les vecteurs de support ?

__Réponse :__ 

#### Affichons la performance du prédicteur

In [ ]:
print(clf.score(X, y))

__Question :__ Quelle mesure de performance est calculée par `clf.score` ? Référez-vous à la [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.score). Que veut dire une performance de 1.0 ?

__Réponse :__

Considérons maintenant les deux classes Versicolour et Virginica !

In [ ]:
clf = svm.SVC(kernel='linear', C=1000)

# on sélectionne les données voulues (2 classes et 2 features)
X = iris.data[iris.target!=0, :2]
print(X.shape)
y = iris.target[iris.target!=0]
print(y.shape)
clf.fit(X, y)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])

# plot support vectors
#ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
#           linewidth=1, facecolors='none')

ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=30,
           linewidth=1, marker='x', color='k')

__Question :__ Où sont situés les vecteurs de support ?

__Réponse :__ 

Quelle est la performance de ce modèle ?

In [ ]:
print(clf.score(X, y))

La performance n'est que de 0.73. Essayons d'utiliser un noyau pour avoir une séparation plus complexe et mieux coller aux données !

### SVM à noyau non linéaire

Nous allons utiliser un noyau RBF gaussien, pour plusieurs valeurs du paramètre gamma. En classe nous avons donné la formule 
$k(x, x') = \frac{1}{\sqrt{2 \pi}} \exp\frac{||x - x'||^2}{2 \sigma^2}$ pour le noyau gaussien. 

__Question :__ À quoi correspond le paramètre gamma ?

__Réponse :__

In [ ]:
# Valeurs de gamma
gamma_range = np.linspace(0.1, 50, 20)

for param in gamma_range:
    clf = svm.SVC(kernel='rbf', C=0.01, gamma=param)
    clf.fit(X, y)
    print("gamma: %.2f" % param, "score: %.2f" % clf.score(X, y))

__Question :__ Affichez maintenant la frontière de séparation pour le dernier de ces classifieurs, qui a un score de 0.80, et donc nettement meilleur que le modèle linéaire précédent (score = 0.73).

In [ ]:
# Réponse : 

__Question :__ Quels points sont vecteurs de supports ? Pensez-vous que le modèle va bien généraliser ?

__Réponse :__

Est-ce que ce modèle se __généralise__ bien, autrement dit, sera-t-il capable de faire de bonnes prédictions sur de nouvelles données que nous n'avons pas utilisées pour le construire ? 

Pour le savoir, nous allons séparer les données en un __jeu d'entraînement__ et un __jeu de test__. Nous allons entraîner nos SVMs sur le jeu d'entraînement seulement, et mesurer leur performance sur le jeu de test. Le jeu de test, étant inconnu au moment de l'entraînement, fait figure de nouvelles données. Pour cela nous allons utiliser la fonction `train_test_split` de scikit-learn, décrite [ici](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.4, random_state=56)

acc_train, acc_test = list(), list()
for param in np.linspace(0.1, 50, 20):
    clf = svm.SVC(kernel='rbf', C=0.01, gamma=param)
    clf.fit(X_train, y_train)
    acc_train.append(clf.score(X_train, y_train))
    acc_test.append(clf.score(X_test, y_test))

In [ ]:
# Afficher le score sur le jeu d'entraînement / le jeu de test pour chaque valeur du paramètre gamma
plt.plot(np.linspace(0.1, 50, 20), acc_train, label='train set')
plt.plot(np.linspace(0.1, 50, 20), acc_test, label='test set')

plt.xlabel("Gamma", fontsize=12)
plt.ylabel("Performance", fontsize=12)

plt.legend(loc='best', fontsize=12)

__Question : __ Observez-vous un effet de surapprentissage ? Où ? 

__Réponse :__  

Le graphique ci-dessus nous donne envie de prendre pour le paramètre gamma une valeur proche de 10 ou 20. Mais attention ! Si nous prenons le paramètre qui marche le mieux sur le dataset de test, nous risquons aussi de surapprendre : nous aurons alors utilisé le jeu de test pour choisir le meilleur modèle, autrement dit, nous aurons touché aux données soi-disant inconnuees lors de l'apprentissage...

Pour éviter cela, nous allons faire une __validation croisée__ (_cross-validation_) sur le jeu d'entraînement.

Nous allons pour cela utiliser la classe [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) du module model_selection de scikit-learn.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Définir les paramètres à tester
parameters = {'kernel':('linear', 'rbf'), 
              'C':[0.1, 1, 10]}

# Initialiser un classifieur SVM
svc = svm.SVC()

# Initialiser une validation croisée
clf = GridSearchCV(svc, parameters)

# Faire tourner la validation croisée sur le jeu d'entraînement
clf.fit(X_train, y_train)

In [ ]:
plt.imshow(clf.cv_results_['mean_test_score'].reshape(len(parameters['kernel']), 
                                                      len(parameters['C'])), interpolation='none')

plt.xlabel('C', fontsize=14)
plt.ylabel('noyau', fontsize=14)
plt.title("Score", fontsize=14)

plt.xticks(np.arange(len(parameters['C'])), parameters['C'], rotation=45)
plt.yticks(np.arange(len(parameters['kernel'])), parameters['kernel'], rotation=45)

plt.colorbar()

__Question :__ Quel est le rôle du paramètre C ? Qu'observez-vous quand C est grand ? 

__Réponse :__

### Mise en pratique : sélectionnons les meilleurs paramètres de SVM pour la classification avec les 4 paramètres disponibles

__Question :__ Entraînez une SVM de classification pour séparer les données sur les deux problèmes (setosa vs virginica, puis versicolor vs virginica). Entraînez votre SVM en validation croisée sur le jeu d'entraînement. Quels paramètres pouvez-vous faire varier ? 

Quelle est la performance de votre modèle optimal _sur le jeu de test_ ? 

##### 1) Setosa vs virginica

In [ ]:
# Réponse

##### 2) Versicolour et Virginica

In [ ]:
# Réponse

__Question bonus :__ Comment construiriez-vous un modèle multi-classe, à base de SVMs, qui assigne une nouvelle observation à une des trois classes setosa, virginica ou versicolor ?